# SQL Troubleshooting Data Issues

## Introduction

**BI users are reporting discrepancies in their reports. Your job is to find out why.**

As a data engineer, you'll frequently encounter situations where:
- 📊 Reports show unexpected numbers
- 🔍 Data doesn't match between different dashboards
- ⚠️ Users complain about missing or incorrect data
- 🐛 Aggregations don't add up correctly

**SQL is your detective tool** to investigate and identify data quality issues.

### Common Data Issues You'll Encounter

1. **Duplicate Records** - Same data appearing multiple times
2. **Missing Values (NULLs)** - Critical fields that should have values but don't
3. **Invalid Values** - Data that doesn't make business sense
4. **Data Type Mismatches** - Numbers stored as text, dates in wrong format
5. **Referential Integrity Issues** - Foreign keys pointing to non-existent records
6. **Inconsistent Data** - Same entity represented differently
7. **Date/Time Issues** - Future dates, dates outside valid ranges
8. **Aggregation Errors** - SUM/COUNT not matching expectations

---

## Scenario: Sales Revenue Report Issues

**The Business Problem:**
The BI team has created a monthly sales revenue dashboard, but users are reporting:
- Total revenue doesn't match the sum of individual transactions
- Some sales appear to be missing
- Revenue numbers fluctuate unexpectedly between refreshes
- Negative revenue values appearing in some reports

**Your Task:**
Use SQL to investigate the `sales_transactions` table and identify all data quality issues.

---

## Step 1: Create Sample Tables and Insert Data

Let's create a sales transactions table and populate it with data that contains various issues.


In [ ]:
-- Create the sales_transactions table
CREATE TABLE sales_transactions (
    transaction_id INT,
    customer_id INT,
    product_id INT,
    transaction_date DATE,
    quantity INT,
    unit_price DECIMAL(10, 2),
    total_amount DECIMAL(10, 2),
    sales_rep_id INT,
    region VARCHAR(50),
    status VARCHAR(20)
);


In [ ]:
-- Insert sample data with various data quality issues
INSERT INTO sales_transactions 
    (transaction_id, customer_id, product_id, transaction_date, quantity, unit_price, total_amount, sales_rep_id, region, status)
VALUES
    -- Normal valid records
    (1, 101, 201, '2024-01-15', 2, 50.00, 100.00, 1, 'North', 'completed'),
    (2, 102, 202, '2024-01-16', 1, 75.50, 75.50, 1, 'South', 'completed'),
    (3, 103, 201, '2024-01-17', 3, 50.00, 150.00, 2, 'East', 'completed'),
    
    -- Issue 1: Missing values (NULLs in critical fields)
    (4, NULL, 203, '2024-01-18', 1, 100.00, 100.00, 2, 'West', 'completed'),
    (5, 104, NULL, '2024-01-19', 2, 25.00, 50.00, 3, 'North', 'completed'),
    (6, 105, 204, NULL, 1, 80.00, 80.00, 1, 'South', 'completed'),
    (7, 106, 205, '2024-01-20', NULL, 60.00, NULL, 2, 'East', 'completed'),
    
    -- Issue 2: Duplicate records (same transaction_id)
    (8, 107, 206, '2024-01-21', 2, 45.00, 90.00, 3, 'West', 'completed'),
    (8, 107, 206, '2024-01-21', 2, 45.00, 90.00, 3, 'West', 'completed'),
    
    -- Issue 3: Calculation mismatch (total_amount doesn't equal quantity * unit_price)
    (9, 108, 207, '2024-01-22', 2, 50.00, 150.00, 1, 'North', 'completed'),  -- Should be 100.00
    (10, 109, 208, '2024-01-23', 3, 30.00, 60.00, 2, 'South', 'completed'),  -- Should be 90.00
    
    -- Issue 4: Invalid values (negative amounts, zero/negative quantities)
    (11, 110, 209, '2024-01-24', -1, 40.00, -40.00, 3, 'East', 'completed'),
    (12, 111, 210, '2024-01-25', 0, 55.00, 0.00, 1, 'West', 'completed'),
    (13, 112, 211, '2024-01-26', 1, -20.00, -20.00, 2, 'North', 'completed'),
    
    -- Issue 5: Future dates (transaction_date in the future)
    (14, 113, 212, '2025-12-31', 2, 70.00, 140.00, 3, 'South', 'completed'),
    (15, 114, 213, '2026-01-01', 1, 65.00, 65.00, 1, 'East', 'completed'),
    
    -- Issue 6: Invalid status values
    (16, 115, 214, '2024-01-27', 2, 45.00, 90.00, 2, 'West', 'pending'),
    (17, 116, 215, '2024-01-28', 1, 80.00, 80.00, 3, 'North', 'cancelled'),
    (18, 117, 216, '2024-01-29', 3, 35.00, 105.00, 1, 'South', 'invalid_status'),
    
    -- Issue 7: Inconsistent region values (case sensitivity, typos)
    (19, 118, 217, '2024-01-30', 2, 50.00, 100.00, 2, 'north', 'completed'),  -- lowercase
    (20, 119, 218, '2024-01-31', 1, 75.00, 75.00, 3, 'Noth', 'completed'),   -- typo
    (21, 120, 219, '2024-02-01', 2, 60.00, 120.00, 1, 'South', 'completed'),
    
    -- Issue 8: Missing total_amount (should be calculated)
    (22, 121, 220, '2024-02-02', 2, 55.00, NULL, 2, 'East', 'completed'),
    (23, 122, 221, '2024-02-03', 1, 90.00, NULL, 3, 'West', 'completed'),
    
    -- Issue 9: Very old dates (outside business range)
    (24, 123, 222, '1990-01-01', 1, 100.00, 100.00, 1, 'North', 'completed'),
    
    -- Issue 10: Duplicate with different values (same transaction_id, different data)
    (25, 124, 223, '2024-02-04', 2, 40.00, 80.00, 2, 'South', 'completed'),
    (25, 125, 224, '2024-02-05', 3, 50.00, 150.00, 3, 'East', 'completed'),  -- Same ID, different data
    
    -- More normal records
    (26, 126, 225, '2024-02-06', 1, 85.00, 85.00, 1, 'North', 'completed'),
    (27, 127, 226, '2024-02-07', 2, 45.00, 90.00, 2, 'South', 'completed');


---

## Step 2: Problem Statement

### The BI Team's Report

**Subject:** Urgent - Sales Revenue Dashboard Data Quality Issues

**Reported Issues:**

1. **Revenue Mismatch:**
   - When we sum all individual transactions, we get $1,850.50
   - But the dashboard shows total revenue as $2,030.50
   - There's a discrepancy of $180.00

2. **Missing Transactions:**
   - Some sales representatives report transactions that don't appear in the dashboard
   - We're missing approximately 3-5 transactions per month

3. **Negative Revenue:**
   - The dashboard occasionally shows negative revenue values
   - This doesn't make business sense for sales transactions

4. **Inconsistent Data:**
   - Revenue numbers change between dashboard refreshes
   - Same date range shows different totals at different times

5. **Status Filter Issues:**
   - When filtering by "completed" status, we still see some transactions that shouldn't be there
   - Some transactions have invalid status values

**Expected Behavior:**
- Total revenue should equal the sum of all completed transactions
- All transactions should have valid customer_id, product_id, and transaction_date
- No negative revenue values
- All status values should be either 'completed', 'pending', or 'cancelled'
- Transaction dates should be within the current year (2024)

**Your Investigation Tasks:**
1. Identify all duplicate transactions
2. Find all records with missing critical values (NULLs)
3. Detect calculation errors (total_amount ≠ quantity × unit_price)
4. Find invalid values (negative amounts, future dates, etc.)
5. Identify inconsistent data (region typos, invalid statuses)
6. Calculate what the correct total revenue should be

---

## Step 3: Troubleshooting Queries

Now let's write SQL queries to investigate each issue.


### Issue 1: Check for Missing Values (NULLs)

Critical fields that should never be NULL:
- `customer_id` - Every transaction must have a customer
- `product_id` - Every transaction must have a product
- `transaction_date` - Every transaction must have a date
- `quantity` - Every transaction must have a quantity
- `total_amount` - Every transaction must have a total


In [ ]:
-- Find all records with NULL values in critical fields
SELECT 
    transaction_id,
    customer_id,
    product_id,
    transaction_date,
    quantity,
    total_amount,
    CASE 
        WHEN customer_id IS NULL THEN 'Missing customer_id'
        WHEN product_id IS NULL THEN 'Missing product_id'
        WHEN transaction_date IS NULL THEN 'Missing transaction_date'
        WHEN quantity IS NULL THEN 'Missing quantity'
        WHEN total_amount IS NULL THEN 'Missing total_amount'
        ELSE 'Multiple NULLs'
    END AS issue_description
FROM sales_transactions
WHERE customer_id IS NULL 
   OR product_id IS NULL 
   OR transaction_date IS NULL 
   OR quantity IS NULL 
   OR total_amount IS NULL;


### Issue 2: Find Duplicate Transactions

Duplicates can cause revenue to be counted multiple times.


In [ ]:
-- Find duplicate transaction_ids
SELECT 
    transaction_id,
    COUNT(*) as duplicate_count
FROM sales_transactions
GROUP BY transaction_id
HAVING COUNT(*) > 1
ORDER BY duplicate_count DESC;


In [ ]:
-- Show the actual duplicate records
SELECT *
FROM sales_transactions
WHERE transaction_id IN (
    SELECT transaction_id
    FROM sales_transactions
    GROUP BY transaction_id
    HAVING COUNT(*) > 1
)
ORDER BY transaction_id;


### Issue 3: Calculation Errors

The `total_amount` should equal `quantity × unit_price`. Let's find mismatches.


In [ ]:
-- Find records where total_amount doesn't match quantity * unit_price
SELECT 
    transaction_id,
    customer_id,
    quantity,
    unit_price,
    total_amount,
    (quantity * unit_price) AS calculated_amount,
    (total_amount - (quantity * unit_price)) AS difference,
    CASE 
        WHEN total_amount != (quantity * unit_price) THEN 'Calculation mismatch'
        ELSE 'OK'
    END AS issue
FROM sales_transactions
WHERE total_amount IS NOT NULL 
  AND quantity IS NOT NULL 
  AND unit_price IS NOT NULL
  AND total_amount != (quantity * unit_price);


### Issue 4: Invalid Values

Check for negative amounts, zero/negative quantities, and negative prices.


In [ ]:
-- Find records with invalid values
SELECT 
    transaction_id,
    customer_id,
    quantity,
    unit_price,
    total_amount,
    CASE 
        WHEN quantity <= 0 THEN 'Invalid quantity (zero or negative)'
        WHEN unit_price < 0 THEN 'Invalid unit_price (negative)'
        WHEN total_amount < 0 THEN 'Invalid total_amount (negative)'
        ELSE 'Multiple issues'
    END AS issue_description
FROM sales_transactions
WHERE quantity <= 0 
   OR unit_price < 0 
   OR total_amount < 0;


### Issue 5: Date Validation

Check for future dates and dates outside valid business range.


In [ ]:
-- Find records with invalid dates
SELECT 
    transaction_id,
    customer_id,
    transaction_date,
    CASE 
        WHEN transaction_date > CURRENT_DATE() THEN 'Future date'
        WHEN transaction_date < '2020-01-01' THEN 'Date too old (before 2020)'
        ELSE 'OK'
    END AS issue_description
FROM sales_transactions
WHERE transaction_date IS NOT NULL
  AND (transaction_date > CURRENT_DATE() OR transaction_date < '2020-01-01');


### Issue 6: Invalid Status Values

Status should only be 'completed', 'pending', or 'cancelled'.


In [ ]:
-- Find records with invalid status values
SELECT 
    transaction_id,
    customer_id,
    status,
    'Invalid status value' AS issue_description
FROM sales_transactions
WHERE status NOT IN ('completed', 'pending', 'cancelled')
   OR status IS NULL;


### Issue 7: Inconsistent Region Values

Check for case sensitivity issues and typos in region names.


In [ ]:
-- Find all unique region values to identify inconsistencies
SELECT 
    region,
    COUNT(*) as transaction_count
FROM sales_transactions
GROUP BY region
ORDER BY region;


In [ ]:
-- Find records with inconsistent region values
SELECT 
    transaction_id,
    customer_id,
    region,
    CASE 
        WHEN LOWER(region) NOT IN ('north', 'south', 'east', 'west') THEN 'Invalid region name'
        WHEN region != INITCAP(region) THEN 'Case inconsistency'
        ELSE 'OK'
    END AS issue_description
FROM sales_transactions
WHERE LOWER(region) NOT IN ('north', 'south', 'east', 'west')
   OR region != INITCAP(region);


### Issue 8: Comprehensive Data Quality Report

Let's create a summary report of all issues found.


In [ ]:
-- Comprehensive data quality check
SELECT 
    transaction_id,
    customer_id,
    product_id,
    transaction_date,
    quantity,
    unit_price,
    total_amount,
    status,
    region,
    CASE 
        WHEN customer_id IS NULL THEN 'Missing customer_id; '
        ELSE ''
    END ||
    CASE 
        WHEN product_id IS NULL THEN 'Missing product_id; '
        ELSE ''
    END ||
    CASE 
        WHEN transaction_date IS NULL THEN 'Missing transaction_date; '
        ELSE ''
    END ||
    CASE 
        WHEN quantity IS NULL THEN 'Missing quantity; '
        ELSE ''
    END ||
    CASE 
        WHEN total_amount IS NULL THEN 'Missing total_amount; '
        ELSE ''
    END ||
    CASE 
        WHEN quantity IS NOT NULL AND unit_price IS NOT NULL AND total_amount IS NOT NULL 
             AND total_amount != (quantity * unit_price) THEN 'Calculation error; '
        ELSE ''
    END ||
    CASE 
        WHEN quantity <= 0 THEN 'Invalid quantity; '
        ELSE ''
    END ||
    CASE 
        WHEN unit_price < 0 THEN 'Invalid unit_price; '
        ELSE ''
    END ||
    CASE 
        WHEN total_amount < 0 THEN 'Invalid total_amount; '
        ELSE ''
    END ||
    CASE 
        WHEN transaction_date > CURRENT_DATE() THEN 'Future date; '
        ELSE ''
    END ||
    CASE 
        WHEN status NOT IN ('completed', 'pending', 'cancelled') THEN 'Invalid status; '
        ELSE ''
    END AS all_issues
FROM sales_transactions
WHERE 
    customer_id IS NULL 
    OR product_id IS NULL 
    OR transaction_date IS NULL 
    OR quantity IS NULL 
    OR total_amount IS NULL
    OR (quantity IS NOT NULL AND unit_price IS NOT NULL AND total_amount IS NOT NULL 
        AND total_amount != (quantity * unit_price))
    OR quantity <= 0 
    OR unit_price < 0 
    OR total_amount < 0
    OR transaction_date > CURRENT_DATE()
    OR status NOT IN ('completed', 'pending', 'cancelled')
ORDER BY transaction_id;


### Issue 9: Revenue Calculation Analysis

Let's calculate what the revenue should be and compare with what's currently in the data.


In [ ]:
-- Calculate revenue with and without duplicates
SELECT 
    'Total from all records (with duplicates)' AS description,
    COUNT(*) AS transaction_count,
    SUM(total_amount) AS total_revenue
FROM sales_transactions
WHERE status = 'completed'

UNION ALL

SELECT 
    'Total from unique transaction_ids (no duplicates)' AS description,
    COUNT(*) AS transaction_count,
    SUM(total_amount) AS total_revenue
FROM (
    SELECT 
        transaction_id,
        total_amount,
        ROW_NUMBER() OVER (PARTITION BY transaction_id ORDER BY transaction_date DESC) AS rn
    FROM sales_transactions
    WHERE status = 'completed'
) AS ranked
WHERE rn = 1

UNION ALL

SELECT 
    'Total from valid records only (no NULLs, no negatives, correct calculations)' AS description,
    COUNT(*) AS transaction_count,
    SUM(CASE 
        WHEN total_amount IS NULL THEN quantity * unit_price
        ELSE total_amount
    END) AS total_revenue
FROM sales_transactions
WHERE status = 'completed'
  AND customer_id IS NOT NULL
  AND product_id IS NOT NULL
  AND transaction_date IS NOT NULL
  AND quantity IS NOT NULL
  AND quantity > 0
  AND unit_price > 0
  AND transaction_id NOT IN (
      SELECT transaction_id
      FROM sales_transactions
      GROUP BY transaction_id
      HAVING COUNT(*) > 1
  );


In [ ]:
-- Detailed breakdown by issue type
SELECT 
    'Missing Values (NULLs)' AS issue_type,
    COUNT(*) AS affected_records
FROM sales_transactions
WHERE customer_id IS NULL 
   OR product_id IS NULL 
   OR transaction_date IS NULL 
   OR quantity IS NULL 
   OR total_amount IS NULL

UNION ALL

SELECT 
    'Duplicate Transactions' AS issue_type,
    COUNT(*) AS affected_records
FROM sales_transactions
WHERE transaction_id IN (
    SELECT transaction_id
    FROM sales_transactions
    GROUP BY transaction_id
    HAVING COUNT(*) > 1
)

UNION ALL

SELECT 
    'Calculation Errors' AS issue_type,
    COUNT(*) AS affected_records
FROM sales_transactions
WHERE total_amount IS NOT NULL 
  AND quantity IS NOT NULL 
  AND unit_price IS NOT NULL
  AND total_amount != (quantity * unit_price)

UNION ALL

SELECT 
    'Invalid Values (Negatives/Zeros)' AS issue_type,
    COUNT(*) AS affected_records
FROM sales_transactions
WHERE quantity <= 0 
   OR unit_price < 0 
   OR total_amount < 0

UNION ALL

SELECT 
    'Invalid Dates' AS issue_type,
    COUNT(*) AS affected_records
FROM sales_transactions
WHERE transaction_date IS NOT NULL
  AND (transaction_date > CURRENT_DATE() OR transaction_date < '2020-01-01')

UNION ALL

SELECT 
    'Invalid Status Values' AS issue_type,
    COUNT(*) AS affected_records
FROM sales_transactions
WHERE status NOT IN ('completed', 'pending', 'cancelled')
   OR status IS NULL;


---

## Step 4: Summary and Next Steps

### Issues Identified

Based on the SQL investigation, we've found:

1. **Missing Values:** Several records have NULL values in critical fields
2. **Duplicates:** Transaction IDs 8 and 25 appear multiple times
3. **Calculation Errors:** Some total_amount values don't match quantity × unit_price
4. **Invalid Values:** Negative quantities, prices, and amounts
5. **Date Issues:** Future dates and very old dates
6. **Invalid Status:** Some records have status values that aren't in the allowed list
7. **Inconsistent Regions:** Case sensitivity and typos in region names

### Recommended Actions

1. **Data Cleaning:**
   - Remove or fix duplicate records
   - Fill in missing values or exclude records with critical NULLs
   - Correct calculation errors
   - Remove or fix invalid values

2. **Data Validation Rules:**
   - Add constraints to prevent future data quality issues
   - Implement data validation in ETL processes
   - Add CHECK constraints for valid ranges

3. **Monitoring:**
   - Set up automated data quality checks
   - Create alerts for data quality issues
   - Regular audits of data integrity

### Key SQL Techniques Used

- `IS NULL` / `IS NOT NULL` - Finding missing values
- `GROUP BY` / `HAVING` - Finding duplicates
- `CASE` statements - Conditional logic for issue identification
- Aggregate functions (`COUNT`, `SUM`) - Summarizing issues
- Comparison operators - Validating data ranges
- `IN` / `NOT IN` - Checking valid values

---

## Practice Exercise

Try to write SQL queries to:

1. Find all transactions that should be included in revenue calculations (valid, completed, no duplicates)
2. Calculate the correct total revenue for January 2024
3. Identify which sales representative has the most data quality issues
4. Create a cleaned version of the data (you can use CTEs or temporary tables)

---

**Remember:** Data quality issues are common in real-world scenarios. As a data engineer, your ability to quickly identify and diagnose these issues using SQL is crucial for maintaining trust in your data systems.
